In [1]:
# requirements.txt should already be installed

## this is a demo notebook to introduce basic langchain abilities

### in this notebook, we will be playing around with langchain for building nerdoscope - a basic horoscope generator for nerds 🤓

In [6]:
# Make sure you pull a model from ollama using `ollama pull <model-name>` (e.g., phi)
from langchain_community.llms import Ollama

llm = Ollama(model="orca-mini:3b")

In [7]:
# Sanity check: Say hey to your local LLM!

llm.invoke("Hey!")

' Hello! How can I help you today?'

In [8]:
# Naively ask for a nerdoscope reading

role = "🧑‍💻 Software Develpoer"
query = f'Please generate a funny "nerdoscope" (horoscope for nerds) message for a {role}. Keep the answer short, 5 sentences max.'

for chunks in llm.stream(query):
    print(chunks, end="", flush=True)

 Hey there fellow nerd! Your day is off to an excellent start with a brainstorming session and a cup of coffee. You're on the right track with your latest project, but just remember to take breaks and not burn yourself out. And if you ever need a good laugh, just try to make time for some video games or watching The Big Bang Theory. Best wishes!

In [16]:
# Let's improve this a bit by using system prompt as context
# This helps the model understand the task better

from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a horoscope bot, that provides a "nerdoscope" (horoscope for nerds) reading for a personnas of the tech industry.'
Given the user's role, create a nerdoscope readings that is funny and relatable to their role.
Keep the answer short, 5 sentences max.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{role}")
])

# We'll now use the prompt and llm to create a chain, this is called LCEL (Langchain Expression Language)
chain = prompt | llm

role = "🧑‍💻 Software Develpoer"
for s in chain.stream({"role": role}):
    print(s, end="", flush=True)

 You are a tech genius, and your life is filled with code and wires. Your nerdoscope reading should be tailored to your specific role as a software developer. Here's one: "Your journey into the coding world is like exploring a vast and mysterious land full of opportunities for innovation and discovery. Be bold and adventurous, and don't be afraid to take risks and try new things!"

In [14]:
# Nice! Let's organize this into a function

from langchain_core.runnables.base import Runnable


def stream_print(chain, context):
    # This function will stream the output of the chain and print it in a readable way for the notebook
    word_count = 0
    buffer = ""

    for s in chain.stream(context):
        buffer += s
        words = buffer.rsplit(' ', 1)
        if len(words) > 1:
            for word in words[:-1]:
                print(word, end=' ', flush=True)
                word_count += 1
                if word_count == 10:
                    print()
                    word_count = 0
            buffer = words[-1]
    if buffer:
        print(buffer, end='', flush=True)
    print()

In [13]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a horoscope bot for personnas in the software development industry.
Given a user's role, generate a personalized "nerdoscope" (horoscope for nerds) message for the week.
Guidelines for the message:
* The message should consists of a single paragraph, 5 sentences max.
* The message should be in the spirit of a horoscope (refer to it as nerdoscope), offering a blend of foresight and advice relevant to the user role, but in a humorous way.
* If possible, the foresight and/or advice should reference a popular tool or technology in the software development industry, relevant to the role.
* The tone should be fun and lighthearted.
* The message should be funny and sarcastic, and personalized to the role.
* The message should include technical jargon.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{role}")
])

chain = prompt | llm

role = "🧑‍💻 Software Develpoer"
stream_print(chain, {"role": role})

 Well, well, well, looks like another week has flown 
by! As a software developer, here's your personalized "nerdoscope" message 
for the week:

"Hi there! I see you're feeling a bit 
stressed out over that project deadline, but don't worry, my 
tech skills can help you out. Just remember, code doesn't 
lie and it'll always find the problem. Cheers to another 
week of coding! #DevOpsGoals"


In [18]:
#  Let's start seeing the power of chains!

from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser, RetryOutputParser, OutputFixingParser
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv

# llm = Ollama(model="orca-mini:3b")

# let's switch to gpt-3.5 hosted in the cloud ☁️🚀
# everything else stays the same!
load_dotenv()
llm = AzureChatOpenAI(
    api_version="2024-02-15-preview",
    azure_deployment="ss_gpt-35-turbo",
)

# let's make the output of nerdoscope more interesting

class Nerdoscope(BaseModel):
    reading: str = Field(description="A nerdoscope reading for the role")
    bugs: int = Field(description="A number of bugs that the user will encounter in the upcoming week")
    coffee_cups: int = Field(description="A number of coffee cups the user will drink in the upcoming week")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Nerdoscope)
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

prompt = PromptTemplate(
    template="{system_prompt}\n{format_instructions}\n{role}",
    input_variables=["role"],
    partial_variables={
        'system_prompt': system_prompt, 
        'format_instructions': parser.get_format_instructions()
        }
)

chain = prompt | llm | parser

role = "🧑‍💻 Software Develpoer"

nerdoscope: Nerdoscope = chain.invoke({"role": role})

print("Hey, here's your nerdoscope for the week:")
print(f"🐞 You will encounter {nerdoscope.bugs} bugs this week.")
print(f"☕️ You will drink {nerdoscope.coffee_cups} coffee cups this week.")
print("🔮", nerdoscope.reading)

Hey, here's your nerdoscope for the week:
🐞 You will encounter 7 bugs this week.
☕️ You will drink 20 coffee cups this week.
🔮 This week, your code will be as flawless as a perfectly optimized algorithm. Bugs will scatter away from your code like frightened little bugs running from a bug zapper. Your debugging skills will be unmatched, and you will squash bugs with the precision of a surgeon. However, be prepared to consume an astronomical amount of coffee to fuel your coding marathons. The caffeine will flow through your veins like electricity through a circuit, keeping you energized and focused. Embrace the bugs and coffee, for they are the fuel that powers your nerdy endeavors.
